In [7]:
import pandas as pd
import numpy as np

In [8]:
df = pd.read_csv("big_mart_data.csv")

In [9]:
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [12]:
nulls = df[df['Item_Weight'].isnull()]['Item_Identifier'].unique()

for col in nulls:
    mode_weight = df[df['Item_Identifier'] == col]['Item_Weight'].mode()
    df.loc[(df['Item_Identifier'] == col) & (df['Item_Weight'].isna()), 'Item_Weight'] = mode_weight

In [16]:
nulls_two = df[df['Item_Weight'].isnull()]['Item_Identifier'].unique().co
nulls_two

AttributeError: 'numpy.ndarray' object has no attribute 'count'

In [6]:
output = []

for col in df.columns:

    null  = np.sum(pd.isna(df[col]))
    unique = df[col].nunique()
    colType = df[col].dtype
    mode = df[col].mode().iloc[0]

    output.append([col, null, unique, colType, mode])

output_df = pd.DataFrame(output, columns = ['column','null values', 'unique', 'dtype', 'mode'])

describe_df = df.describe().transpose() 
combine_df = pd.concat([ output_df.set_index('column'), describe_df], axis = 1, join='outer')

combine_df

,null values,unique,dtype,mode,count,mean,std,min,25%,50%,75%,max
Item_Identifier,0,1559,object,FDG33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Item_Weight,1463,415,float64,12.15,7060.0,12.857645,4.643456,4.555,8.773750,12.600000,16.850000,21.350000
Item_Fat_Content,0,5,object,Low Fat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Item_Visibility,0,7880,float64,0.0,8523.0,0.066132,0.051598,0.000,0.026989,0.053931,0.094585,0.328391
Item_Type,0,16,object,Fruits and Vegetables,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Item_MRP,0,5938,float64,172.0422,8523.0,140.992782,62.275067,31.290,93.826500,143.012800,185.643700,266.888400
Outlet_Identifier,0,10,object,OUT027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Outlet_Establishment_Year,0,9,int64,1985,8523.0,1997.831867,8.371760,1985.000,1987.000000,1999.000000,2004.000000,2009.000000
Outlet_Size,2410,3,object,Medium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Outlet_Location_Type,0,3,object,Tier 3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
